# Voting Mechanism Experiment Template

## Experiment #1 

(Use a unique identifier for the number next to "Experiment".)

## Experiment Overview

### Key Question

In Single-Choice Weighted Plurality, does adding weight for each NFTs produce different results (final winner) than one wallet, one vote?

### Procedure for Generating Data

A **trial** consists of the following steps:
**Step 1:** We load the actual TE Academy data available from the Otterspace subgraph, accessed on May 28. This gives us the real data available to voters. 

**Step 2:** We create two different weightings for the voters.
A. *One Wallet, One Vote:* each voter is assigned a weight of 1.0, regardless of how many NFTs they hold. (This is also *uniform weighting*.)
B. *Each NFT has a weight of 1.0*: each voter is assigned a weight equal to the number of NFTs they hold. Each NFT is worth 1.0.

**Step 3:** We randomly assign voter choices.

**Step 4:** We calculate the results (including final winner) for each weighting, and keep track of results in a DataFrame. 

We aim to calculate 100_000 trials. 

### Measurement

We measure which proportion of the time the two weightings produced the same winner. 

## Code for Simulation

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import os
from random import choice
from typing import Dict

import sys
sys.path.append('..')  # Add this line to include the directory above

# Custom imports
from custom_types import UserNFTs
from mechanisms.single_choice_weighted_plurality import SingleChoiceWeightedPlurality
from mechanisms.group_hug_mechanism import GroupHug

## Importing the Data 

In the cell below, we are importing the available data as of May 28, 2024 regarding which users have which NFTs. 

In [2]:
file_name = "../data/nft_data_may_28_2024_cleaned.csv"
file_exists = os.path.exists(file_name)
file_exists

True

# Key Question

How often would a vote that incorporates credibility-weighting by NFTs produce a different outcome than simply doing one wallet, one vote? We use the actual TE Academy NFT data as of May 28 to look at this question. 

## Inputting Sample Data 

In the cells below, we load the data available from the `.csv` file in the `VOTER_DATA_FILENAME`. 

If you wish to use a different `.csv` file, change this line. 

In [3]:
VOTER_DATA_FILENAME = "../data/nft_data_may_28_2024_cleaned.csv" # Adjust data here. 

voter_data = pd.read_csv(VOTER_DATA_FILENAME) #Convert to a Pandas DataFrame. 

# Sometimes there is a 
if ['Unnamed: 0'] in voter_data.columns:
    voter_data.drop(columns = ['Unnamed: 0'], 
                    inplace = True)
voter_data.set_index('ID', inplace = True)

In [27]:
sample_voters = voter_data.to_dict(orient='index')
sample_voters

In [29]:
# NOTE: This next step isn't strictly necessary at this stage. 
voters = {key: UserNFTs(sample_voters.get(key))
          for key, _ in sample_voters.items()
          }

In [30]:
voters

{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 0,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 0,
  'SPEAKER_BARCAMP_PARIS_23': 0,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 0,
  'FUND_MOD1': 0,
  'FUND_MOD2': 0,
  'FUND_MOD3': 1,
  'FUND_MOD4': 1,
  'FUND_MOD5': 0},
 '0x049debbad61a20e21e872b06ef4f25be1253c802': {'NFTREP_V1': 0,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 1,
  'FUND_AUTHOR': 0,
  'STUDY_GROUP_HOST_360_22': 0,
  'STUDY_GROUP_HOST_FUND_22_23': 0,
  'SPEAKER_BARCAMP_PARIS_23': 0,
  'BARCAMP_PARIS_23': 0,
  'TEAM_BARCAMP_PARIS_23': 0,
  'FUND_MOD1': 0,
  'FUND_MOD2': 0,
  'FUND_MOD3': 1,
  'FUND_MOD4': 1,
  'FUND_MOD5': 1},
 '0xa481db1ac55683dfb2847c02e417b7e411bcbbea': {'NFTREP_V1': 1,
  'SPEAKER_ETHCC_PARIS23': 0,
  'STUDY_GROUP_HOST_C2_22_23': 0,
  'ETHCC_23': 0,
  'FUND_AUTHOR': 0,
  'STUDY_GR

When working with data, it's often helpful to see a particular entry to see what it looks like. 

In [10]:
first_voter = list(sample_voters.values())[0]

### Assigning Weights

We need to create a dictionary of weights for the NFTS. 
The dictionary `NFT_weights` gives a weight of 1.0 to each NFT held by current TEA owners.  

In [11]:
NFT_weights = {key: 1.0 
               for key
               in first_voter.keys()
               }
NFT_weights

{'NFTREP_V1': 1.0,
 'SPEAKER_ETHCC_PARIS23': 1.0,
 'STUDY_GROUP_HOST_C2_22_23': 1.0,
 'ETHCC_23': 1.0,
 'FUND_AUTHOR': 1.0,
 'STUDY_GROUP_HOST_360_22': 1.0,
 'STUDY_GROUP_HOST_FUND_22_23': 1.0,
 'SPEAKER_BARCAMP_PARIS_23': 1.0,
 'BARCAMP_PARIS_23': 1.0,
 'TEAM_BARCAMP_PARIS_23': 1.0,
 'FUND_MOD1': 1.0,
 'FUND_MOD2': 1.0,
 'FUND_MOD3': 1.0,
 'FUND_MOD4': 1.0,
 'FUND_MOD5': 1.0}

Single Choice Weighted Plurality requires that voters have some kind of weight. 
The function below takes a dictionary of voters with NFTs held, and a dictionary of weights.
It returns the weight that each voter should hold, based on 

In [12]:
def calc_voter_weights_from_NFT_weight(voters: Dict,
                                       nft_weights: Dict) -> Dict:
    """
    Given a list of voters, the NFTs they hold, and the 
    """
    new_dict = {}
    for voter in voters.keys():
        new_dict[voter] = {}
        new_dict[voter]["weight"] = 0
        for nft_name, nft_val in nft_weights.items():
            if voters.get(voter).get(nft_name):
                new_dict[voter]["weight"] += nft_val

    return new_dict

Now we use the function to assign each voter a weight, based on the NFTs. 

In [13]:
voter_weights = calc_voter_weights_from_NFT_weight(voters,
                                                   NFT_weights)
voter_weights

As a comparison, we also create a version where each voter is assigned the same weight of 1.0, regardless of how many NFTs they hold. 

In [15]:
uniform_weights = {voter: {}"weight": 1.0} for voter in sample_voters.keys()}
uniform_weights

## Data Generation

The data generation is below. Each voter is randomly assigned a choice to vote for. 
The results of using SingleChoiceWeightedPlurality are calculated for each trial, based on these voter choices and weights. 
These results are stored in a DataFrame. 

In [17]:
# Create the mechanism
SCWPCalculator = SingleChoiceWeightedPlurality()

# Set the number of experiments
NUM_EXPERIMENTS = 10_000

# Create an empty DataFrame to store simulation results
results_list = [] 

for k in range(NUM_EXPERIMENTS):
    # Generate choices for each voter
    sample_choices = {
                   key: choice(["A","B","C","D"])
                   for key in sample_voters.keys()
                 }
    
    # Calculate who would win with weight of 1 per each NFT
    weighted_winner, weighted_results  = SCWPCalculator.calculate(voter_weights,
                            sample_choices)
    # Calculate who would win with uniform weight 
    uniform_winner, uniform_results = SCWPCalculator.calculate(uniform_weights,
                            sample_choices)
                   
    # Create an empty DataFrame to store simulation results
    results_list.append({'Experiment': k,
                        'weighted_winner': weighted_winner, 
                        'weighted_candidate_scores': weighted_results,
                        'uniform_winner': uniform_winner,
                        'uniform_candidate_scores': uniform_results,
                        "votes": sample_choices
                        }
                        )


results_df = pd.DataFrame(results_list)

#TODO: Refactor for speed if needed.  

Let's see what the data looks like. 

In [18]:
results_df

,Experiment,weighted_winner,weighted_candidate_scores,uniform_winner,uniform_candidate_scores,group_hug_winner,group_hug_results,votes
0,0,A,"{'C': 241.0, 'D': 209.0, 'B': 160.0, 'A': 251.0}",A,"{'C': 90.0, 'D': 87.0, 'B': 74.0, 'A': 96.0}",C,"{'C': 26, 'B': 26, 'A': 24, 'D': 24}",{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87':...
1,1,B,"{'A': 174.0, 'B': 246.0, 'D': 246.0, 'C': 195.0}",B,"{'A': 78.0, 'B': 96.0, 'D': 96.0, 'C': 77.0}",B,"{'A': 20, 'B': 30, 'C': 24, 'D': 26}",{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87':...
2,2,C,"{'C': 230.0, 'A': 224.0, 'D': 206.0, 'B': 201.0}",C,"{'C': 93.0, 'A': 82.0, 'D': 81.0, 'B': 91.0}",C,"{'C': 33, 'B': 16, 'A': 26, 'D': 25}",{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87':...
3,3,B,"{'B': 264.0, 'C': 198.0, 'A': 177.0, 'D': 222.0}",B,"{'B': 101.0, 'C': 77.0, 'A': 76.0, 'D': 93.0}",D,"{'C': 25, 'B': 21, 'A': 25, 'D': 29}",{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87':...
4,4,A,"{'C': 222.0, 'D': 209.0, 'A': 250.0, 'B': 180.0}",D,"{'C': 77.0, 'D': 100.0, 'A': 95.0, 'B': 75.0}",A,"{'C': 21, 'B': 21, 'A': 33, 'D': 25}",{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87':...
...,...,...,...,...,...,...,...,...
9995,9995,D,"{'A': 209.0, 'B': 215.0, 'D': 219.0, 'C': 218.0}",B,"{'A': 77.0, 'B': 94.0, 'D': 82.0, 'C': 94.0}",B,"{'A': 21, 'B': 31, 'C': 31, 'D': 18}",{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87':...
9996,9996,A,"{'A': 226.0, 'B': 213.0, 'D': 220.0, 'C': 202.0}",D,"{'A': 87.0, 'B': 84.0, 'D': 88.0, 'C': 88.0}",C,"{'A': 22, 'B': 24, 'C': 28, 'D': 25}",{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87':...
9997,9997,D,"{'D': 239.0, 'C': 203.0, 'B': 211.0, 'A': 208.0}",D,"{'D': 94.0, 'C': 82.0, 'B': 88.0, 'A': 83.0}",B,"{'C': 26, 'B': 28, 'A': 18, 'D': 28}",{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87':...
9998,9998,C,"{'C': 247.0, 'B': 213.0, 'A': 212.0, 'D': 189.0}",C,"{'C': 94.0, 'B': 93.0, 'A': 80.0, 'D': 80.0}",B,"{'C': 21, 'B': 30, 'A': 30, 'D': 19}",{'0xbb8743ea733155fe5e81ed285aea72cc19b2ca87':...


## Measurements

We calculate what proportion of the time the two methods gave the same result. 

In [19]:
# Calculate proportion where they agreed. 

(results_df['weighted_winner'] == results_df['uniform_winner']).mean()

0.6755